# Measuring Production in the North Atlantic Ocean using BGC-Argo Oxygen Measurements

This notebook will attempt to follow the work of Johnson and Bif (_in prep_, doi:) to calculate production in the North Atlantic Ocean by aggregating data from many BGC-Argo floats to analyze them as a single timeseries.

The process can be summarized as:

1. Collect all oxygen surface data (_P_ < 25dbar)
2. Calculate the oxygen anomaly (observed oxygen - saturation concentration)
3. Find the local time of day for that data (i.e. convert from UTC to local time)
4. Filter data (as described below)

A key piece to performing this analysis is that the floats are not only sampling at one time of day. In other words, if a float is on a 10-day cycle period and therefore always surfaces at the same time every 10 days (ex. always at roughly 6:00AM) then the analysis will not work as the data will be skewed. For this reason, Johnson and Bif introduced a selection criteria where the expected number of profiles for each hour was calculated (Nprof/24), and if the actual number of profiles for each hour exceeded the expected number by more than a factor of 3, the float was removed from the analysis. Additionally, any data points that exceeded 6 times the standard deviation of all anomaly data were removed to avoid large outliers.

For our purposes, we will select data in the North Atlantic, that have oxygen sensors, and then perform the above filtering. We'll do this using the `argopy` package:


In [1]:
import argopy
argopy.set_options(
    mode='expert', src='localftp', dataset='bgc',
    local_ftp='/Users/gordonc/Documents/data/Argo/'
)

Now we define out North Atlantic region with a bounding box and get the profiles in that region:


In [2]:
nw_atlantic = [-62, -36, 52, 65]
ix_fetcher = argopy.IndexFetcher(ds='bgc')
nw_index = ix_fetcher.region(nw_atlantic).to_dataframe()

The `argopy` module doesn't handle the BGC index yet, so we'll do a workaround to get a list of  BGC floats:

In [3]:
import bgcArgoDMQC as bgc
bgc_ix = bgc.get_index() # default is bgc index

# get the floats in the region
bgc_in_region = nw_index[nw_index.wmo.isin(bgc_ix.wmo)]
# the bgc index in that region
bgc_ix_in_region = bgc_ix[bgc_ix.wmo.isin(bgc_in_region.wmo)]
# final index with just DOXY floats
final_index = bgc_ix_in_region[['DOXY' in p for p in bgc_ix_in_region.parameters]].reset_index()